<a href="https://colab.research.google.com/github/dragoon-lzt/Bayesian-Theory-and-Computation/blob/main/HW/HW1/HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

## Problem 2.
### (5)
simulation: 

In [ ]:
# one-step transition function
def transfunc(i):
  if i==0:
    a = np.random.rand()
    if a < 0.5:
      return 0
    else:
      return 1
  else:
    a = np.random.rand()
    if a>0.5:
      return i-1
    elif a<1/(2*(i+1)):
      return i+1
    else:
      return i

np.random.seed(1024)

X0=1

X = np.zeros(10000)
X[0] = transfunc(X0)
for t in range(1,10000):
  X[t] = transfunc(X[t-1])
mu = np.mean(X)
sigma = np.var(X) 

print("mean     = ", mu)
print("variance = ", sigma)

mean     =  0.9698
variance =  0.92548796


## Problem 3.
